In [217]:
import os
import sys
import time
import pandas as pd
import asyncio
from binance import Client, AsyncClient, BinanceSocketManager
from binance.exceptions import BinanceAPIException
from pprint import pformat
from binance.helpers import round_step_size
from binance.enums import *

In [218]:
TEST_API_KEY = 'Your test API Key'
TEST_API_SECRET = 'Your test API Secret'


In [219]:
#  Instantiate Binance API client TESTNET US Market
client = Client(api_key=TEST_API_KEY, api_secret=TEST_API_SECRET, testnet=True, tld='us')

In [220]:
def convert_scientific_to_decimal(num):
    if 'e' in str(num):
        return "{:.15f}".format(float(str(num)))
    else:
        return str(num)

In [221]:
symbol_info = {}
def get_symbol_info(symbols):
    global symbol_info
    for symbol in symbols:
        info = client.get_symbol_info(symbol)
        base_asset = info['baseAsset']

        #  PRICE_FILTER
        price_filter_min_quantity = float(info['filters'][0]['minPrice'])
        price_filter_max_quantity = float(info['filters'][0]['maxPrice'])
        price_filter_tick_size = float(info['filters'][0]['tickSize'])
        
        #  LOT_SIZE (for market- and limit orders)
        lot_size_min_quantity = float(info['filters'][2]['minQty'])
        lot_size_max_quantity = float(info['filters'][2]['maxQty'])
        lot_size_step_size = float(info['filters'][2]['stepSize'])
        
        #  MIN NOTIONAL
        min_notional = float(info['filters'][3]['minNotional'])
        

        symbol_info[symbol] = {'price_filter_min_quantity': price_filter_min_quantity, 'price_filter_max_quantity': price_filter_max_quantity,\
                               'price_filter_tick_size': price_filter_tick_size,'lot_size_min_quantity': lot_size_min_quantity, \
                                'lot_size_min_quantity': lot_size_min_quantity, 'lot_size_max_quantity': lot_size_max_quantity, \
                               'lot_size_step_size': lot_size_step_size, 'min_notional': min_notional, }
        
symbols = ['BTCUSDT']
get_symbol_info(symbols)
print(pformat(symbol_info))
print(pformat(symbol_asset_map))

{'BTCUSDT': {'lot_size_max_quantity': 900.0,
             'lot_size_min_quantity': 1e-06,
             'lot_size_step_size': 1e-06,
             'min_notional': 10.0,
             'price_filter_max_quantity': 1000000.0,
             'price_filter_min_quantity': 0.01,
             'price_filter_tick_size': 0.01}}
{'BTCUSDT': 'BTC', 'ETHBTC': 'ETH'}


In [222]:
#  Get the balances for both coins
btc_balance = client.get_asset_balance(asset='BTC').get('free')
print(btc_balance)
usdt_balance = client.get_asset_balance(asset='USDT').get('free')
print(usdt_balance)

1.00659400
9510.51720147


In [223]:
#  What's the exchange rate between BTC and USDT? 
price_btc = float(client.get_symbol_ticker(symbol='BTCUSDT')['price'])
print('One BTC can buy ',price_btc,' USDT')

One BTC can buy  40485.34  USDT


In [224]:
# How many BTC do we want to spend? Let's say $0.01 worth
btc_quantity = float(0.01 / price_btc)
btc_quantity = convert_scientific_to_decimal(btc_quantity)
print(btc_quantity)

0.000000247002989


In [226]:
#  Create a Buy Order
order = client.create_order(
    symbol='BTCUSDT',
    quantity=btc_quantity,
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
)

BinanceAPIException: APIError(code=-1111): Precision is over the maximum defined for this asset.

In [227]:
#  Fix precision issue - too many decimals
btc_quantity = '{:.8f}'.format(float(btc_quantity))
print(btc_quantity)

#  Create a Buy Order
order = client.create_order(
    symbol='BTCUSDT',
    quantity=btc_quantity,
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
)

0.00000025


BinanceAPIException: APIError(code=-1013): Filter failure: LOT_SIZE

In [228]:
#  Check quantity filters
info = symbol_info['BTCUSDT']
min_quantity = info['lot_size_min_quantity']
max_quantity = info['lot_size_max_quantity']
print('min_quantity: ',min_quantity,', max_quantity: ',max_quantity,', btc_quantity: ',btc_quantity)
btc_quantity = float(btc_quantity)
if btc_quantity < min_quantity:
    print('Quantity too small')
if btc_quantity > max_quantity:
    print('Quantity too large')


min_quantity:  1e-06 , max_quantity:  900.0 , btc_quantity:  0.00000025
Quantity too small


In [229]:
#  Increase quantity to $0.1
btc_quantity = float(0.1 / price_btc)
btc_quantity = '{:.8f}'.format(float(btc_quantity))
print(btc_quantity)

if float(btc_quantity) < min_quantity:
    print('Quantity too small')


order = client.create_order(
    symbol='BTCUSDT',
    quantity=btc_quantity,
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
)

0.00000247


BinanceAPIException: APIError(code=-1013): Filter failure: LOT_SIZE

In [230]:
#  Need to round up quantity to meet step size
step_size = info['lot_size_step_size']
print('step_size: ',step_size)
btc_quantity = round_step_size(btc_quantity, step_size)
btc_quantity = '{:.6f}'.format(float(btc_quantity))
print('btc_quantity', btc_quantity)

order = client.create_order(
    symbol='BTCUSDT',
    quantity=btc_quantity,
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
)

step_size:  1e-06
btc_quantity 0.000002


BinanceAPIException: APIError(code=-1013): Filter failure: MIN_NOTIONAL

In [231]:
# The Min Notional filter defines the minimum value calculated in the quote asset for a symbol.
# For our symbol BTCUSDT the quote symbol is USDT, so the min notional values is in USDT.
# notional_value = price * quantity
min_notional = info['min_notional']
my_notional = price_btc * float(btc_quantity)
print(f"Min Notional {min_notional}, My notional: {my_notional}")
if my_notional < min_notional:
    print('Notional is too small')
    
#  To fix this, we need to again increase our initial investment amount
#  Let's try $10
btc_quantity = float(15 / price_btc)
btc_quantity = round_step_size(btc_quantity, step_size)
btc_quantity = '{:.6f}'.format(float(btc_quantity))
#print('btc_quantity: ', btc_quantity)
my_notional = price_btc * float(btc_quantity)
print(f"Min Notional {min_notional}, My notional: {my_notional}")
if my_notional < min_notional:
    print('Notional is too small')

Min Notional 10.0, My notional: 0.08097067999999999
Notional is too small
Min Notional 10.0, My notional: 14.9795758


In [232]:
#  Create a Buy Order
order = client.create_order(
    symbol='BTCUSDT',
    quantity=btc_quantity,
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
)
print(pformat(order))

{'clientOrderId': 'f2MQvNZg4Ga8brgpYzbFcz',
 'cummulativeQuoteQty': '14.99483460',
 'executedQty': '0.00037000',
 'fills': [{'commission': '0.00000000',
            'commissionAsset': 'BTC',
            'price': '40526.58000000',
            'qty': '0.00037000',
            'tradeId': 1358283}],
 'orderId': 4325008,
 'orderListId': -1,
 'origQty': '0.00037000',
 'price': '0.00000000',
 'side': 'BUY',
 'status': 'FILLED',
 'symbol': 'BTCUSDT',
 'timeInForce': 'GTC',
 'transactTime': 1650065965156,
 'type': 'MARKET'}


In [233]:
#  Get the balances for both coins
btc_balance = client.get_asset_balance(asset='BTC').get('free')
print(btc_balance)
usdt_balance = client.get_asset_balance(asset='USDT').get('free')
print(usdt_balance)


1.00696400
9495.52236687
